In [1]:
import numpy as np, PIL.Image as Image, matplotlib.pyplot as plt, os, pandas as pd, datasets, torch

In [2]:
celeba = datasets.CelebA(subset = 'validation')

In [3]:
image, label = celeba[0]

In [4]:
label

{'face_recognition': -1,
 'emotion': -1,
 'age': -1,
 'gender': -1,
 'race': -1,
 'attributes': tensor([-1, -1,  1, -1, -1,  1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
          1,  1, -1, -1, -1, -1,  1,  1, -1,  1, -1, -1, -1,  1, -1,  1, -1, -1,
          1, -1, -1,  1], dtype=torch.int8),
 'pose': tensor([-999., -999., -999.])}

In [6]:
from utility_scripts.pose_estimation_utilities import euler_angles_to_matrix, matrix_to_euler_angles

In [7]:
import torch

In [17]:
angles = torch.tensor(data = [[90, 90, 45]])
rotation_matrix = euler_angles_to_matrix(angles)
rotation_matrix

tensor([[[-3.0909e-08,  1.2803e-08,  1.0000e+00],
         [ 7.0711e-01,  7.0711e-01,  1.2803e-08],
         [-7.0711e-01,  7.0711e-01, -3.0909e-08]]])

In [18]:
matrix_to_euler_angles(rotation_matrix)

tensor([[90., 90., 45.]])